<a href="https://colab.research.google.com/github/wlsdl021498/201701199/blob/main/%EC%98%81%ED%99%94_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D%EA%B8%B0_%ED%88%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 환경 준비

!pip install konlpy
import pandas as pd

from IPython.display import clear_output
clear_output()

print("###준비가 완료되었습니다.###")

###준비가 완료되었습니다.###


In [ ]:
#@title 데이터 불러오기(탭 \t 으로 분리)
구글드라이브ID = '1LFI4k6__9RNxRXWfS_hf0mIPpPH4Pkhy'  #@param {type: "string"}
파일저장이름 = 'rawdata.txt'  #@param {type: "string"}

import time
start = time.time() # 시작시간

# 구글 드라이브 연결을 위한 기본 세팅
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 문서 ID로 실제 파일 불러오는 법
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
rawdata_downloaded = drive.CreateFile({'id': 구글드라이브ID})
rawdata_downloaded.GetContentFile(파일저장이름)

print(파일저장이름+"을 정상적으로 가져왔습니다.")

import pandas as pd
# 파일을 DataFrame으로 전환
rawdata = pd.read_csv(파일저장이름, sep="\t", header=0, encoding='utf-8', engine='python')
rawdata.head()

rawdata.txt을 정상적으로 가져왔습니다.


,type,text
0,네이버,"퍼시픽림보고 느낀건데, 트랜스포머1 만한건 다신 못볼것같다.."
1,네이버,시간이 흐를수록 깨닫게 된다. 로봇영화가 계속해서 나오지만 트랜스포머1만큼의 감동은...
2,네이버,CG의 한획을 그었지
3,네이버,영화관에서 못본게 한이된다
4,네이버,이 영화 보고 처음 걸어나왔을때 주변에 있는 차들이 다 변신할것만 같았는데..


In [ ]:
#@title 형태소 분석 및 저장(파이썬은 0부터 시작!!!)
텍스트데이터위치 =   1#@param {type: "number"}
타입데이터위치 =   0#@param {type: "number"}
파일저장이름 = '\uD2B8\uB79C\uC2A4\uD3EC\uBA381.csv'  #@param {type: "string"}

# 분석하기
from itertools import product
import numpy as np
from konlpy.tag import Okt
okt = Okt()

segments  = []
for index, row in rawdata.iterrows():
    textdata = row[텍스트데이터위치]
    type = row[타입데이터위치]
    words = list(okt.pos(textdata))
    words = pd.DataFrame.from_records(words)
    words['type'] = type
    words['count'] = 1      
    segments.append(words)

# 단일 데이터프레임 구조로 변환
framemerge = pd.concat(segments)

# 데이터프레임의 컬럼값 조정
framemerge.columns = ["terms", "pos", "type", "count"]

print("##################################################")
print("타입별 형태소 추출 결과")
print(framemerge)
print("##################################################")

# 데이터 정리하기
dfwordflag = framemerge.groupby(['terms', 'pos', "type"])['count'].sum()

# 단일 데이터프레임 구조로 변환
dfwordflag = pd.DataFrame(dfwordflag)

print("##################################################")
print("형태소+품사+타입을 기준으로 묶어주기 결과")
print(dfwordflag)
print("##################################################")


#csv로 출력
dfwordflag.to_csv(파일저장이름,header='true', quotechar='"', encoding='utf-8')

print("##################################################")
print("데이터를 " + 파일저장이름 + "로 저장 완료")
print("##################################################")


##################################################
타입별 형태소 추출 결과
    terms          pos type  count
0    퍼시픽림         Noun  네이버      1
1      보고         Noun  네이버      1
2    느낀건데         Verb  네이버      1
3       ,  Punctuation  네이버      1
4   트랜스포머         Noun  네이버      1
..    ...          ...  ...    ...
11     지만         Josa  네이버      1
12   리메이크         Noun  네이버      1
13      한   Determiner  네이버      1
14     거래         Noun  네이버      1
15     니까         Josa  네이버      1

[7505 rows x 4 columns]
##################################################
##################################################
형태소+품사+타입을 기준으로 묶어주기 결과
                        count
terms pos         type       
!     Punctuation 네이버      15
!!    Punctuation 네이버       6
!!!   Punctuation 네이버       6
!!!!  Punctuation 네이버       2
!!.   Punctuation 네이버       1
...                       ...
히어로   Noun        네이버       1
힘     Noun        네이버       1
힘들    Adjective   네이버       2
힘들듯   Adjective   네이버       1
힘듬  